# Config

> Configuration dataclasses for card stack initialization and visual styling.

In [ ]:
#| default_exp core.config

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import Tuple

from cjm_fasthtml_tailwind.utilities.effects import ring, shadow
from cjm_fasthtml_tailwind.core.base import combine_classes
from cjm_fasthtml_daisyui.utilities.semantic_colors import shadow_dui, ring_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

## Prefix Counter

Auto-generates unique prefixes (`cs0`, `cs1`, ...) for card stack instances
when no explicit prefix is provided. Ensures multi-instance HTML ID uniqueness.

In [ ]:
#| export
_prefix_counter: int = 0

def _auto_prefix() -> str:  # Unique prefix string (e.g., "cs0", "cs1")
    """Generate an auto-incrementing unique prefix."""
    global _prefix_counter
    p = f"cs{_prefix_counter}"
    _prefix_counter += 1
    return p

def _reset_prefix_counter() -> None:
    """Reset the prefix counter (for testing only)."""
    global _prefix_counter
    _prefix_counter = 0

In [ ]:
# Test auto-prefix generation
_reset_prefix_counter()
assert _auto_prefix() == "cs0"
assert _auto_prefix() == "cs1"
assert _auto_prefix() == "cs2"
_reset_prefix_counter()
assert _auto_prefix() == "cs0"  # Reset works
print("Prefix counter tests passed!")

Prefix counter tests passed!


## Style Defaults

Module-level constants for focused card emphasis. Computed from the
`cjm_fasthtml_tailwind` and `cjm_fasthtml_daisyui` libraries so
consumers can see (and override) the exact default class strings.

In [ ]:
#| export
_DEFAULT_FOCUS_RING: str = combine_classes(ring(1), ring_dui("color-mix(in_oklch,var(--color-primary),transparent_50%)"))
_DEFAULT_FOCUS_SHADOW: str = combine_classes(shadow.lg, shadow_dui.primary)
_DEFAULT_FOCUS_BORDER_RADIUS: str = str(border_radius.box)

In [ ]:
# Verify default class strings
assert _DEFAULT_FOCUS_RING == "ring-1 ring-[color-mix(in_oklch,var(--color-primary),transparent_50%)]"
assert _DEFAULT_FOCUS_SHADOW == "shadow-lg shadow-primary"
assert _DEFAULT_FOCUS_BORDER_RADIUS == "rounded-box"
print("Style default constants tests passed!")

Style default constants tests passed!


## CardStackStyleConfig

Visual styling for a card stack instance. Spacing values are set as CSS
custom properties on the outer container and consumed by child elements
via Tailwind's arbitrary value syntax (e.g., `gap-[var(--cs0-section-gap)]`).
Consumer JS can update them dynamically via `style.setProperty()`.

In [ ]:
#| export
@dataclass
class CardStackStyleConfig:
    """Visual styling for a card stack instance."""
    # Spacing (CSS length values — set as CSS custom properties for dynamic JS adjustment)
    section_gap: str = "1rem"           # Gap between cards in each section
    slot_padding: str = "0.25rem"       # Padding around context card content
    viewport_padding_x: str = "0.5rem"  # Horizontal outer container padding
    viewport_padding_y: str = "0.5rem"  # Vertical outer container padding
    focus_padding_x: str = "0.5rem"     # Horizontal focused section padding
    focus_padding_b: str = "1rem"       # Bottom focused section padding

    # Visual emphasis (CSS class strings)
    focus_ring: str = _DEFAULT_FOCUS_RING                    # Ring classes for focused card
    focus_shadow: str = _DEFAULT_FOCUS_SHADOW                # Shadow classes for focused card
    focus_border_radius: str = _DEFAULT_FOCUS_BORDER_RADIUS  # Border radius class for focused card

    def css_vars_style(
        self,
        prefix: str,  # Card stack instance prefix
    ) -> str:  # Inline style string with CSS custom property declarations
        """Generate CSS custom property declarations as an inline style string."""
        return (
            f"--{prefix}-section-gap: {self.section_gap}; "
            f"--{prefix}-slot-padding: {self.slot_padding}; "
            f"--{prefix}-viewport-padding-x: {self.viewport_padding_x}; "
            f"--{prefix}-viewport-padding-y: {self.viewport_padding_y}; "
            f"--{prefix}-focus-padding-x: {self.focus_padding_x}; "
            f"--{prefix}-focus-padding-b: {self.focus_padding_b}"
        )

In [ ]:
# Test CardStackStyleConfig defaults
style = CardStackStyleConfig()
assert style.section_gap == "1rem"
assert style.slot_padding == "0.25rem"
assert style.viewport_padding_x == "0.5rem"
assert style.viewport_padding_y == "0.5rem"
assert style.focus_padding_x == "0.5rem"
assert style.focus_padding_b == "1rem"
assert style.focus_shadow == "shadow-lg shadow-primary"
assert style.focus_border_radius == "rounded-box"
print("CardStackStyleConfig defaults tests passed!")

CardStackStyleConfig defaults tests passed!


In [ ]:
# Test css_vars_style generates correct CSS custom property declarations
style = CardStackStyleConfig()
css = style.css_vars_style("cs0")
assert "--cs0-section-gap: 1rem" in css
assert "--cs0-slot-padding: 0.25rem" in css
assert "--cs0-viewport-padding-x: 0.5rem" in css
assert "--cs0-viewport-padding-y: 0.5rem" in css
assert "--cs0-focus-padding-x: 0.5rem" in css
assert "--cs0-focus-padding-b: 1rem" in css
print("css_vars_style tests passed!")

css_vars_style tests passed!


In [ ]:
# Test css_vars_style with custom values and different prefix
style = CardStackStyleConfig(section_gap="0.5rem", slot_padding="0.125rem")
css = style.css_vars_style("chat")
assert "--chat-section-gap: 0.5rem" in css
assert "--chat-slot-padding: 0.125rem" in css
# Unchanged defaults still present
assert "--chat-viewport-padding-x: 0.5rem" in css
print("css_vars_style custom values test passed!")

css_vars_style custom values test passed!


## CardStackConfig

Initialization-time settings for a card stack instance. Created once and reused
across requests. The `prefix` drives `CardStackHtmlIds` and `CardStackButtonIds`
generation for multi-instance support.

In [ ]:
#| export
@dataclass
class CardStackConfig:
    """Initialization-time settings for a card stack instance."""
    prefix: str = field(default_factory=_auto_prefix)  # HTML ID prefix (auto-generated if omitted)

    # Card count selector options
    visible_count_options: Tuple[int, ...] = (1, 3, 5, 7, 9)  # Choices for card count dropdown
    auto_visible_count: bool = True  # Whether "Auto" option appears in card count dropdown

    # Width slider bounds
    card_width_min: int = 30    # Width slider minimum (rem)
    card_width_max: int = 120   # Width slider maximum (rem)
    card_width_step: int = 5    # Width slider step (rem)

    # Scale slider bounds
    card_scale_min: int = 50    # Scale slider minimum (%)
    card_scale_max: int = 200   # Scale slider maximum (%)
    card_scale_step: int = 10   # Scale slider step (%)

    # Interaction
    click_to_focus: bool = False  # Whether context cards get transparent click overlay
    disable_scroll_in_modes: Tuple[str, ...] = ()  # Mode names where scroll-to-nav is suppressed

    # Visual styling
    style: CardStackStyleConfig = field(default_factory=CardStackStyleConfig)  # Visual styling config

In [ ]:
# Test CardStackConfig defaults with auto-prefix
_reset_prefix_counter()
config1 = CardStackConfig()
config2 = CardStackConfig()
assert config1.prefix == "cs0"
assert config2.prefix == "cs1"
assert config1.prefix != config2.prefix  # Unique prefixes
print("Auto-prefix uniqueness tests passed!")

Auto-prefix uniqueness tests passed!


In [ ]:
# Test CardStackConfig with explicit prefix
config = CardStackConfig(prefix="text-stack")
assert config.prefix == "text-stack"
print("Explicit prefix test passed!")

Explicit prefix test passed!


In [ ]:
# Test CardStackConfig field defaults
_reset_prefix_counter()
config = CardStackConfig()
assert config.visible_count_options == (1, 3, 5, 7, 9)
assert config.auto_visible_count == True
assert config.card_width_min == 30
assert config.card_width_max == 120
assert config.card_width_step == 5
assert config.card_scale_min == 50
assert config.card_scale_max == 200
assert config.card_scale_step == 10
assert config.click_to_focus == False
assert config.disable_scroll_in_modes == ()
assert isinstance(config.style, CardStackStyleConfig)
assert config.style.section_gap == "1rem"
print("CardStackConfig defaults tests passed!")

CardStackConfig defaults tests passed!


In [ ]:
# Test CardStackConfig with custom values
config = CardStackConfig(
    prefix="chat",
    visible_count_options=(3, 5, 7),
    card_width_min=40,
    card_width_max=100,
    card_scale_min=75,
    card_scale_max=150,
    click_to_focus=True,
    disable_scroll_in_modes=("split", "edit"),
    style=CardStackStyleConfig(section_gap="0.5rem", focus_shadow="shadow-md"),
)
assert config.prefix == "chat"
assert config.visible_count_options == (3, 5, 7)
assert config.card_width_min == 40
assert config.card_scale_min == 75
assert config.click_to_focus == True
assert config.disable_scroll_in_modes == ("split", "edit")
assert config.style.section_gap == "0.5rem"
assert config.style.focus_shadow == "shadow-md"
assert config.style.slot_padding == "0.25rem"  # Unchanged default
print("CardStackConfig custom value tests passed!")

CardStackConfig custom value tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()